In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2023-05-11 11:57:02.973930: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
df = pd.read_csv("training.csv")

In [5]:
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
17995,im having ssa examination tomorrow in the morn...,0
17996,i constantly worry about their fight against n...,1
17997,i feel its important to share this info for th...,1
17998,i truly feel that if you are passionate enough...,1


In [6]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df["text"], df["label"], test_size=0.2)

In [7]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_texts)

In [8]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [9]:
maxlen = 50
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post', truncating='post')

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=50, input_length=maxlen),
    tf.keras.layers.GRU(units=64, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(units=5, activation='softmax')
])

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

df = pd.read_csv("training.csv")

train_texts, test_texts, train_labels, test_labels = train_test_split(df["text"], df["label"], test_size=0.2)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)


maxlen = 50
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post', truncating='post')

num_classes = len(np.unique(train_labels))
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=50, input_length=maxlen),
    tf.keras.layers.GRU(units=64, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_padded, train_labels, epochs=10, batch_size=32, validation_data=(test_padded, test_labels))

test_loss, test_accuracy = model.evaluate(test_padded, test_labels)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)


Epoch 1/10
450/450 [==============================] - 13s 24ms/step - loss: 1.5869 - accuracy: 0.3328 - val_loss: 1.5767 - val_accuracy: 0.3411
Epoch 2/10
450/450 [==============================] - 10s 23ms/step - loss: 1.5786 - accuracy: 0.3333 - val_loss: 1.5796 - val_accuracy: 0.3411
Epoch 3/10
450/450 [==============================] - 11s 24ms/step - loss: 1.5766 - accuracy: 0.3365 - val_loss: 1.5795 - val_accuracy: 0.3411
Epoch 4/10
450/450 [==============================] - 10s 23ms/step - loss: 1.2718 - accuracy: 0.4172 - val_loss: 0.9561 - val_accuracy: 0.5736
Epoch 5/10
450/450 [==============================] - 10s 23ms/step - loss: 0.4620 - accuracy: 0.8399 - val_loss: 0.2717 - val_accuracy: 0.8886
Epoch 6/10
450/450 [==============================] - 11s 24ms/step - loss: 0.2149 - accuracy: 0.9192 - val_loss: 0.2066 - val_accuracy: 0.9119
Epoch 7/10
450/450 [==============================] - 11s 24ms/step - loss: 0.1509 - accuracy: 0.9428 - val_loss: 0.1946 - val_accuracy:

In [17]:
from sklearn.metrics import classification_report

In [18]:
test_predictions = model.predict(test_padded)
test_pred_labels = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(test_labels, axis=1)
print(classification_report(test_true_labels, test_pred_labels))

113/113 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1018
           1       0.96      0.93      0.95      1228
           2       0.79      0.86      0.82       278
           3       0.99      0.89      0.94       501
           4       0.81      0.91      0.86       440
           5       0.75      0.79      0.77       135

    accuracy                           0.92      3600
   macro avg       0.88      0.89      0.88      3600
weighted avg       0.93      0.92      0.92      3600

